## Optimized Query

>  🚨 Note!
> The given query contained several syntax issues, including misspelled column names and typographical errors, such as missing commas in the `SELECT` statement. I did not consider these basic syntax fixes as part of the optimizations I made

In [ ]:
-- Creating indexes for optimization
CREATE INDEX idx_clickup_name ON ClickUp(Name);
CREATE INDEX idx_float_name ON Float(Name);
CREATE INDEX idx_clickup_hours ON ClickUp(hours);

-- Optimized query
SELECT
    c.Name,
    f.Role,
    SUM(c.hours) AS Total_Tracked_Hours,
    SUM(f.Estimed_Hours) AS Total_Allocated_Hours
FROM
    ClickUp c
JOIN
    Float f
    ON c.Name = f.Name
WHERE
    c.hours > 0   
GROUP BY
    c.Name,
    f.Role
HAVING
    SUM(c.hours) > 100
ORDER BY
    Total_Allocated_Hours DESC;


### Explanation

In [ ]:
GROUP BY
    c.Name,
    f.Role,
    c.Date

-- OR

MAX(c.Date) AS Latest_Date ---> Not necessary

- Since the `date` column isn't in the `GROUP BY` clause, I would have included it in the `GROUP BY` clause or replaced it with `MAX(c.Date)`, This makes the query simpler and keeps it logically correct. 

But I prefered removing unnecessary columns in the `GROUP BY` clause or using `MAX(c.Date)` in the SELECT clause if it's not essential to my output.

In [ ]:
-- Index on `Name` in ClickUp and Flaot tables to optimize the JOIN operation
CREATE INDEX idx_clickup_name ON ClickUp(Name);
CREATE INDEX idx_float_name ON Float(Name);

-- Index on `hours` in ClickUp table to optimize the aggregation and filtering in HAVING clause
CREATE INDEX idx_clickup_hours ON ClickUp(hours);

- I prefer indexing the `Name` column in both the `Clickup` and `Float` tables and the `hours` column on the `Clickup` table to improve the efficiency of the `JOIN` operation.  

    The `Name` indexes will ensure that the database can efficiently match rows between the `ClickUp` and `Float` tables during the `JOIN` operation, also index on `hours` in the `ClickUp` table helps optimize the `SUM(c.hours)` aggregation and makes the filtering in the `HAVING` clause faster.

    basically since indexing is like an index in a book, it helps the database quickly locate the data without having to scan the entire table 

    **Reason for choosing Indexing over partition:**
    Indexing is generally more beneficial for tables that are not exceedingly large, where the overhead of maintaining partitions might not be justified and it also increases complexity in terms of table management, especially when adding or removing partitions.


## The Performance Result:

>  🚨 Note!
> I used a **copy** of the data ingested to my Postgres Database from the ETL process to carry out the optimization tests, and I used `Dbeaver` as my Database Management tool

<div style="display: flex; justify-content: space-around; align-items: center;">
    <figure style="text-align: center; margin: 0 10px;">
        <img src="../static/Optimized-shot.png" alt="Optimized Query" width="500" />
    </figure>
    <figure style="text-align: center; margin: 0 10px;">
        <img src="../static/not_optimized-shot.png" alt="Not Optimized" width="500" />
    </figure>
</div>

##                          Optimized query (Left Image) | Not Optimized Query (Right Image)
1) **Execution Time**
    Without Optimization: Execution time is 4.7 ms.
    With Optimization: Execution time is 2.067 ms.

    The optimized query is almost half the execution time of the non optimized one, which is a significant improvement.

2)  **Cost**
    Without Optimization: The total cost is 301.00..303.36.
    With Optimization: The total cost is 89.05..89.13.

    The cost reduction from 301 to 89 shows that the query planner chose a more efficient execution plan with the optimizations. Lower cost generally indicates better resource utilization (e.g., CPU, memory, I/O).

3) **Without Optimization**
    Sort Key: `sum(f.estimated_hours)` is done using the quicksort method with a memory usage of 25kB.
    The sort takes a longer time with higher cost.
    With Optimization:

    The sort is done more efficiently with the same quicksort method, but it is faster with reduced cost and time for sorting (2.032 ms vs. 4.655 ms).

4) **Join Operation**
    Without Optimization: The Hash Join has a cost of 16.98..176.73, and it's joining over 3992 rows.

    With Optimization: The Hash Join now costs 1.29..73.75, and the row count is 2924.

    The join cost has dropped significantly, and fewer rows are being processed. This could be due to optimizations in filtering, indexing, or better hash join management

5) **Filter (Rows Removed)**
    Without Optimization: The filter `(SUM(c.hours) > 100)` removes 316 rows.
    With Optimization: The filter removes only 1 row.

    This indicates that the query optimizer is now effectively filtering unnecessary rows earlier in the query execution, reducing the number of rows that need to be processed in later stages.

>  🚨 Note! <br>
> The dataset used is relatively small and a dataset with a larger size would have been more ideal to carry out performance optimiztions